In [1]:
# Import library
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [2]:
print("Versi scikit-learn yang terinstal:", sklearn.__version__)

Versi scikit-learn yang terinstal: 1.2.2


dataset diambil dari website Fatsecret Indonesia : https://www.fatsecret.co.id/Diary.aspx?pa=aja

In [3]:
df = pd.read_excel("dataset_workout.xlsx")
df

,Aktivitas,kategori,kal/jam
0,tidur,ringan,66
1,istirahat,ringan,74
2,duduk,ringan,103
3,kerjaan kantoran (dibalik meja),ringan,110
4,mengemudi,ringan,118
...,...,...,...
158,ski (cross country),berat,882
159,tinju,berat,882
160,lari - 13 km/jam,berat,992
161,"lari - 14,5 km/jam",berat,1102


In [4]:
# Encode kategori
df_encoded = pd.get_dummies(df['kategori'])

In [5]:
# Gabungkan data yang sudah diencode dengan dataframe awal
df = pd.concat([df, df_encoded], axis=1)
df

,Aktivitas,kategori,kal/jam,berat,moderat,ringan
0,tidur,ringan,66,0,0,1
1,istirahat,ringan,74,0,0,1
2,duduk,ringan,103,0,0,1
3,kerjaan kantoran (dibalik meja),ringan,110,0,0,1
4,mengemudi,ringan,118,0,0,1
...,...,...,...,...,...,...
158,ski (cross country),berat,882,1,0,0
159,tinju,berat,882,1,0,0
160,lari - 13 km/jam,berat,992,1,0,0
161,"lari - 14,5 km/jam",berat,1102,1,0,0


In [6]:
# Pisahkan data training dan testing
X = df.drop(['Aktivitas', 'kategori'], axis=1)
y = df['kategori']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Pilih model machine learning (Random Forest Classifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [9]:
# Latih model
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
# Evaluasi model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model: {accuracy * 100:.2f}%')

Akurasi model: 100.00%


In [19]:
# Fungsi untuk merekomendasikan aktivitas berdasarkan kategori
def recommend_activity(category):
    # Ubah kategori menjadi one-hot encoding
    input_data = pd.get_dummies(pd.Series(category))

    # Lengkapi data dengan kolom dummy yang mungkin hilang
    missing_cols = set(X.columns) - set(input_data.columns)
    for col in missing_cols:
        input_data[col] = 0

    # Sesuaikan urutan kolom agar sesuai dengan model yang telah dilatih
    input_data = input_data[X.columns]

    # Prediksi kategori
    predicted_category = model.predict(input_data)[0]

    # Filter aktivitas berdasarkan kategori
    recommended_activities = df[df['kategori'] == predicted_category][['Aktivitas', 'kal/jam']]

    # Reset index agar tidak mengikutinya
    recommended_activities_df = recommended_activities.reset_index(drop=True)

    return recommended_activities_df

Untuk inputnya

bisa dari weight loss (ada di inputnya punya ikmal yang food)

1.   'ringan' = 'Maintain Weight'
2.   'moderat' = 'Mild Weight Loss'
3.   'moderat' = 'Weight Loss'
4.   'berat' = 'Extreme Weight Loss'







In [23]:
# Contoh penggunaan
kategori_input = 'moderat'
rekomendasi = recommend_activity(kategori_input)
print(f'Rekomendasi aktivitas untuk kategori {kategori_input}: {rekomendasi}')

Rekomendasi aktivitas untuk kategori moderat:                               Aktivitas  kal/jam
0                         mencuci mobil      220
1                             memancing      220
2                              berlayar      220
3                           berselancar      220
4                        boat atau kano      220
5                               pilates      228
6          berjalan (sedang) - 5 km/jam      243
7              senam (ringan, di rumah)      257
8       berjalan (latihan) - 5,5 km/jam      279
9       bersepeda (santai) - <16 km/jam      294
10                              berkuda      294
11                         menyapu daun      294
12                          motor-cross      294
13                      golf (berjalan)      294
14               bersepeda di dalam air      294
15                              bowling      294
16                            bola voli      294
17                  tari (gerak lambat)      294
18           pekerjaan 

In [24]:
# Simpan model
joblib.dump(model, 'DailyActivitiesAndCaloriesRecommendation.joblib')

['DailyActivitiesAndCaloriesRecommendation.joblib']

In [25]:
# Load model dari file joblib
loaded_model = joblib.load('DailyActivitiesAndCaloriesRecommendation.joblib')

In [26]:
import joblib
import jsonpickle

# Konversi model ke dalam format JSON menggunakan jsonpickle
json_model = jsonpickle.encode(loaded_model)

# Simpan model dalam format JSON ke dalam file
with open('model_3.json', 'w') as json_file:
    json_file.write(json_model)